# LCEL
LCEL是一种声明式的方式，它能够轻松地将各种步骤组合在一起。LCEL从设计之初就支持将原型快速投入生产使用，从最简单的“prompt + LLM”链到最复杂的链条（有人成功地在生产环境中运行了包含数百个步骤的LCEL链）。以下是LCEL的一些优势：
1. 流式支持：LCEL使用一种流式处理的机制，将原始的语言模型输出（tokens）通过流式传输直接传送到一个解析器，得到即时的输出。

2. 异步支持：使用LCEL构建的任何链条都可以使用同步API（比如原型设计时在Jupyter笔记本中使用）和异步API（比如在LangServe服务器中使用）进行调用。这使您能够在原型和生产环境中使用相同的代码。

3. 并行执行：当您的LCEL链条具有可以并行执行的步骤时（例如，从多个检索器中获取文档），系统会自动执行并行操作，无论是在同步还是异步接口中，以获得最小的延迟。

4. 重试和回退（Retries and fallbacks）：您可以为LCEL链条的任何部分配置重试和回退机制。这是一种在大规模情境下提高链条可靠性的好方法。

5. 访问中间结果：对于更复杂的处理链条，能够在最终输出生成之前访问中间步骤的结果通常非常有用。比如告知最终用户某些操作正在进行中，或调试链条。中间结果可以流式传输，且这种功能在LangServe服务器上是可用的。这意味着无论您在何处运行您的LCEL链条，都可以访问中间结果，增强了其灵活性和普适性。

6. 输入和输出模式：输入和输出模式（schemas）为每个LCEL链条提供了基于您链条结构推断的Pydantic和JSONSchema模式，也就是可以根据您链条的结构自动生成数据模式，确保数据的准确性和一致性。

7. LangSmith跟踪集成：随着链条变得越来越复杂，了解每个步骤发生了什么变得越来越重要。使用LCEL，所有步骤都会自动记录到LangSmith，以获得最大的可观察性和调试能力。

8. LangServe部署集成：使用LCEL创建的任何链条都可以轻松地通过LangServe进行部署。

总的来说，LCEL提供了一种灵活、高效且可靠的方式来构建和管理复杂的处理链条，使您能够轻松地在不同环境中使用相同的代码，并且具有高度的可观察性和可调试性。


In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

## Runnable
LCEL提供了声明式的方式来组合Runnables成为链。它是一个标准接口，可以轻松定义自定义链条并以标准方式调用它们，还可以批处理、流处理等。该标准接口包括以下几个方法（前缀'a'表示为对应的异步处理方法）：

- `invoke/ainvoke`：处理单个输入
- `batch/abatch`：批处理多个输入列表
- `stream/astream`：流式处理单个输入并产生输出
- `astream_log`：流式返回中间步骤的数据，以及最终响应数据
- `astream_events`:在链中发生的beta流事件(在langchain-core 0.1.14中引入)

该接口的存在使得创建和调用自定义链条变得更加简单，并提供了一致性的调用方式，无论是同步还是异步处理。另外，这些方法的输入类型和输出类型因组件而异：


| Component | Input Type | Output Type |
| --- | --- | --- |
| Prompt | Dictionary | PromptValue |
| ChatModel | Single string, list of chat messages or a PromptValue | ChatMessage |
| LLM | Single string, list of chat messages or a PromptValue | String |
| OutputParser | The output of an LLM or ChatModel | Depends on the parser |
| Retriever | Single string | List of Documents |
| Tool | Single string or dictionary, depending on the tool | Depends on the tool |

 所有可运行的组件都提供了输入和输出模式langchain.schema，以便检查它们的输入和输出结构。另外所有方法（invoke、batch、stream等）都可以接受一个可选的config参数，用于配置执行、添加标签、元数据等。

input_schema：一个由可运行组件结构自动生成的输入 Pydantic 模型
output_schema：一个由可运行组件结构自动生成的输出 Pydantic 模型

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")

prompt

ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me a joke about {topic}'))])

In [3]:
chain = prompt | model

### Input Schema
可运行对象接受的输入的描述。这是从任何Runnable的结构动态生成的Pydantic模型。您可以对其调用`.schema()`以获得JSONSchema表示。

In [4]:
# The input schema of the chain is the input schema of its first part, the prompt.
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [5]:
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'topic': {'title': 'Topic', 'type': 'string'}}}

In [6]:
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {

### Output Schema
Runnable产生的输出的描述。这是从任何Runnable的结构动态生成的Pydantic模型。您可以对其调用`.schema()`以获得JSONSchema表示。

In [7]:
# The output schema of the chain is the output schema of its last part, in this case a ChatModel, which outputs a ChatMessage
chain.output_schema.schema()

{'title': 'ChatOpenAIOutput',
 'anyOf': [{'$ref': '#/definitions/AIMessage'},
  {'$ref': '#/definitions/HumanMessage'},
  {'$ref': '#/definitions/ChatMessage'},
  {'$ref': '#/definitions/SystemMessage'},
  {'$ref': '#/definitions/FunctionMessage'},
  {'$ref': '#/definitions/ToolMessage'}],
 'definitions': {'ToolCall': {'title': 'ToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'object'},
    'id': {'title': 'Id', 'type': 'string'}},
   'required': ['name', 'args', 'id']},
  'InvalidToolCall': {'title': 'InvalidToolCall',
   'type': 'object',
   'properties': {'name': {'title': 'Name', 'type': 'string'},
    'args': {'title': 'Args', 'type': 'string'},
    'id': {'title': 'Id', 'type': 'string'},
    'error': {'title': 'Error', 'type': 'string'}},
   'required': ['name', 'args', 'id', 'error']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'proper

## Runnable组合方式
常用的Runnable组合方式有：
- `RunnableSequence`：顺序执行，后一个Runnable的输入来自前一个的输出，可以通过|操作符|或者传递Runnable列表来构建
- `RunnableParallel`：并行执行，每个Runnable拿到同样的输入，可以在一个序列内使用字典字面量或者传入字典来构建。

1. 构建简单的PromptTemplate + ChatModel链条

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = prompt | model

2. 使用RunnableSequence和RunnableParallel来组合Runnable构建链。

In [9]:
from langchain_core.runnables import RunnableSequence,  RunnableLambda
sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
res = sequence.invoke(1)
print(res)
res = sequence.batch([1,2,3])
print(res)

4
[4, 6, 8]


In [10]:
sequence = RunnableLambda(lambda x: x + 1) | {
    "model1": RunnableLambda(lambda x: x * 2),
    "model2": RunnableLambda(lambda x: x * 3)
}
res = sequence.invoke(1)
print(res)
res = sequence.batch([1,2,3])
print(res)

{'model1': 4, 'model2': 6}
[{'model1': 4, 'model2': 6}, {'model1': 6, 'model2': 9}, {'model1': 8, 'model2': 12}]


上述代码中，RunnableLambda是一个用来把普通函数包装成Runnable的工具。 第二个sequence演示了如何通过一个dict字面量内嵌RunnableParallel，从而并发执行x + 1后面的两种运算。invoke结果是一个dict，带有每个并发Runnable的结果。总结一下：

- RunnableLambda将普通函数包装成Runnable
- 通过|操作符可以将多个Runnable组合成序列
- 序列内也可以通过字典字面量并发执行多个Runnable


### 修改行为

另外，Runnable可以修改其行为，如添加重试、日志、可配置化（2.2节）、启用debug等，还可以可以通过回调函数来跟踪调试（2.3节）。这些方法适用于任何 Runnable，包括各种组合而成的Runnable，例如：

In [20]:
from langchain_core.runnables import RunnableSequence,  RunnableLambda

import random

def add_one(x: int) -> int:
    print("**************", x)
    return x + 1

def buggy_double(y: int) -> int:
    '''有70%概率会失败的错误代码''' 
    if random.random() > 0.3:
        print('代码执行失败,可能会重试!')
        raise ValueError('触发了错误代码')
    return y * 2

sequence = RunnableLambda(add_one) | RunnableLambda(buggy_double).with_retry(
        stop_after_attempt=10,   # 最多重试10次
        wait_exponential_jitter=False # 重试间隔没有扰动
        )

In [21]:
print(sequence.input_schema.schema())	    # 打印推断出的输入模式

{'title': 'add_one_input', 'type': 'integer'}


In [22]:
print(sequence.output_schema.schema())	    # 打印推断出的输出模式 

{'title': 'buggy_double_output', 'type': 'integer'}


In [24]:
print(sequence.invoke(2))	

************** 2
6


述代码中，使用RunnableLambda把两个函数包装成Runnable，通过运算符|组装成一个序列sequence。另外为buggy_double函数额外添加了重试机制。

 随着链越来越长，能够看到中间结果以调试和跟踪链会很有用。您可以将全局调试标志设置为 True，以启用所有链的调试输出：

In [25]:
from langchain.globals import set_debug
set_debug(True)


In [27]:
from langchain_community.callbacks.tracers import ConsoleCallbackHandler #回调
chain.invoke(2, config={'callbacks': [ConsoleCallbackHandler()]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "input": 2
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "input": 2
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me a joke about 2"
  ]
}
[llm/end] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] [1.30s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Why was 6 afraid of 7? Because 7 8 (ate) 9!",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
     

AIMessage(content='Why was 6 afraid of 7? Because 7 8 (ate) 9!', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 14, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-b3c8b8f7-6db9-4207-aa47-f793000b974a-0')

## 输入输出模式
###  前置知识：Pydantic

Pydantic 是一个 Python 库，用于数据验证和设置数据模型。它提供了一个简单而强大的方式来定义数据模型、验证数据的结构和类型，并且支持自动生成模型。Pydantic 的核心目标是帮助开发者轻松地定义数据模型并确保数据的有效性。其核心特性为：

1. 声明性数据验证：Pydantic 允许您使用 Python 的声明性语法定义数据模型，包括字段名、字段类型和约束。这些模型是基于标准的 Python 类定义的，通过类型提示和特定的字段配置来实现。

2. 自动化数据验证：一旦定义了数据模型，Pydantic 可以自动验证数据是否符合模型的预期结构和类型。它会检查输入数据是否满足指定的字段和约束，并进行自动类型转换和校验。

3. 自动生成模型：Pydantic 能够根据已有的 Python 类自动生成数据模型，省去了手动编写模型的繁琐过程。这使得在构建复杂数据结构时更加方便快捷。


In [28]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    username: str
    email: str
    age: int

在这个示例中，User 类继承自 BaseModel，并定义了四个字段：id、username、email 和 age，分别对应整数类型、字符串类型和整数类型。这个模型指定了每个字段的类型，它们是必需的，并且是字符串类型的 email 字段还会按照电子邮件地址的格式进行验证。

In [29]:
user_data = {
    "id": 1,
    "username": "john_doe",
    "email": "john@example.com",
    "age": 30
}

# 创建用户实例并验证数据
user = User(**user_data)
print(user)


id=1 username='john_doe' email='john@example.com' age=30


在这个例子中，我们创建了一个字典 user_data，然后使用 User 模型将其转换为一个用户实例 user。Pydantic 会自动验证数据是否符合模型的结构和类型，并创建一个符合模型的用户对象。如果数据不符合模型要求，Pydantic 将会引发一个 ValidationError 异常，指示数据无效。

  这样，Pydantic 提供了一种简单而强大的方式来定义数据模型，并验证数据的正确性，使得开发者可以更加轻松地处理和验证数据。

In [32]:
set_debug(False)

### Stream

In [8]:
for s in chain.stream({"topic": "bears"}):
    print(s.content, end="", flush=True)

Why did the bear break up with his girlfriend? 

Because he couldn't bear the relationship any longer!

### Invoke

In [30]:
chain.invoke({"topic": "bears"})


[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "topic": "bears"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "topic": "bears"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me a joke about bears"
  ]
}
[llm/end] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] [1.45s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Why did the bear bring a flashlight to the party? \n\nBecause he heard it was going to be a \"beary\" good time!",
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
   

AIMessage(content='Why did the bear bring a flashlight to the party? \n\nBecause he heard it was going to be a "beary" good time!', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 13, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-847d6163-ca32-41ed-aa6c-910322693145-0')

### Batch

In [31]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])



[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "topic": "bears"
}
[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "topic": "cats"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "topic": "bears"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[chain/start] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "topic": "cats"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me a joke about bears"
  ]
}
[llm/start] [1:chain:RunnableSequence > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: tell me a joke about cats"
  ]
}
[llm/end] [1:chain:RunnableSequen

[AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-da303fcf-f0dc-4bb3-b7ea-fa5d0b72a38c-0'),
 AIMessage(content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-d6687061-71f0-4f04-871a-dd270d6b6a86-0')]

您可以使用 max_concurrency 参数来设置并发请求的数量。

In [33]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}], config={"max_concurrency": 5})


[AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-47f88b05-07d1-401e-acfa-5dbac1fc1b1f-0'),
 AIMessage(content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-3e4bf412-1643-4d9c-9ee7-9f0a97ccdc9c-0')]

## 异步调用

### 单个输入并行处理
让我们看一下 LCEL如何支持并行请求。例如，当使用 RunnableParallel（通常表示为一个字典）时，它会并行执行每个元素。

In [34]:
from langchain_core.runnables import RunnableParallel

chain1 = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
chain2 = (
    ChatPromptTemplate.from_template("write a short (2 line) poem about {topic}")
    | model
)
combined = RunnableParallel(joke=chain1, poem=chain2)
chain1.invoke({"topic": "bears"})



AIMessage(content='Why did the bear bring a flashlight to the party? \n\nBecause he heard it was going to be a "beary" good time!', response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 13, 'total_tokens': 41}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'stop', 'logprobs': None}, id='run-5cf80853-a29f-48f5-85c0-c1d78a112e52-0')

In [38]:
%%time
chain2.invoke({"topic": "bears"})


CPU times: total: 62.5 ms
Wall time: 3.3 s


AIMessage(content='In the forest they roam, strong and free\nBears, majestic creatures of mystery', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 17, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-1607afb8-21e8-489e-b4a8-0894d1939681-0')

In [37]:
%%time
combined.invoke({"topic": "bears"})


CPU times: total: 62.5 ms
Wall time: 4.33 s


{'joke': AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-eabe5a1b-918e-4b4e-b699-e9dfd93bc59c-0'),
 'poem': AIMessage(content='In the forest they roam, strong and free\nMajestic creatures, wild and full of mystery', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 17, 'total_tokens': 37}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-c2f029c2-08f2-473c-ba42-5e08310cb54d-0')}

### batch输入并行处理

In [39]:
%%time
combined.batch([{"topic": "bears"}, {"topic": "cats"}])

CPU times: total: 109 ms
Wall time: 3.63 s


[{'joke': AIMessage(content="Why did the bear break up with his girlfriend? \n\nBecause he couldn't bear the relationship any longer!", response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 13, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-2ad7f08a-d8fb-42be-9b1b-f7d0f37cb05e-0'),
  'poem': AIMessage(content='In the forest they roam, strong and free\nBears, majestic creatures of mystery', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 17, 'total_tokens': 34}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-601fa441-690e-4052-a8d9-7e94fec0a973-0')},
 {'joke': AIMessage(content='Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!', response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 13, 'total_tokens': 33